In [1]:
# 載入所需套件
import cv2
import time
import datetime
import imutils

import smtplib
from email.mime.text import MIMEText

import json

# 使用外部json導入smtp帳密
account = json.load(open('account.json', 'r', encoding='utf-8'))

gmail_user = account['user']
gmail_password = account['password']

html = """
<html>
    <head>
        <style>
            h1{
                color: #ff5e5e;
            }
            a{
                border-color: #0366d6;
                text-decoration: none;
                background-color: #0366d6;
                border: solid 1px #0366d6;
                border-radius: 5px;
                color: #ffffff;
                display: inline-block;
                font-size: 16px;
                font-weight: bold;
                margin: 0;
                padding: 10px 20px;
            }
            .footer{
                text-align: center;
                color: #888;
                border-color: #999;
            }
        </style>
    </head>
    <body>
        <h1>📢有人入侵!</h1>
        <img src="https://i.imgur.com/x4Red6D.png">
        <a href="https://github.com/junyou1998/AI-guard">此為開源專案 點此瞭解更多❤</a>
        
        <div class="footer">
            <hr>
            Copyright © 2020 JunYou X 吳沅騏. All rights reserved
            <hr>
        </div>
        
    </body>
</html>
"""

msg = MIMEText(html,'html')
msg['Subject'] = 'AI 保全'
msg['From'] = gmail_user
msg['To'] = account['to_user']

# 寄件函式
def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.send_message(msg)
    server.quit()
    print('email sent!')

# 動作偵測函式
def motion_detection():
    video_capture = cv2.VideoCapture(0)
    #等待鏡頭準備
    time.sleep(2)

    first_frame = None 
    first = None
    
    #不斷抓取每一幀
    while True:
        
        frame = video_capture.read()[1] 
        text = 'Unoccupied'
        
        #將每一幀轉為灰階
        greyscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        #使用高斯模糊
        gaussian_frame = cv2.GaussianBlur(greyscale_frame, (21,21),0)
        blur_frame = cv2.blur(gaussian_frame, (5,5)) 
       
        greyscale_image = blur_frame 

        #建立比較樣本
        if first_frame is None:
            first_frame = greyscale_image            
        else:
            pass

        frame = imutils.resize(frame, width=1000)
        #計算兩個圖像中每個元素/像素之間的差，樣本-灰度
        frame_delta = cv2.absdiff(first_frame, greyscale_image)
        #判斷房間明暗，將大於100的值，轉為白色(255)
        thresh = cv2.threshold(frame_delta, 100, 255, cv2.THRESH_BINARY)[1]
        #形態學操作 膨脹
        dilate_image = cv2.dilate(thresh, None, iterations=2)
        
        #尋找輪廓
        cnt = cv2.findContours(dilate_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        

        for c in cnt:
            #當變化量大於臨界值(800)
            if cv2.contourArea(c) > 800:
                
                #繪製動態框
                (x, y, w, h) = cv2.boundingRect(c) 
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2) 
                #狀態文字
                text = 'Occupied'
         
                if first is None:
                    send_mail()#寄發信件
                    first = datetime.datetime.now()
                    print(datetime.datetime.now())
                    
                record = (datetime.datetime.now()-first).seconds     
                    
                if(record>60):
                    send_mail()#寄發信件
                    first = datetime.datetime.now()
                    print('超過1分鐘了')
                else:
                    pass
                    
                print(record)
            else:
                pass
                
        font = cv2.FONT_HERSHEY_SIMPLEX 
        
        #房間狀態浮水印
        cv2.putText(frame, 'Room Status: %s' % (text), 
            (10,20), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (0, 0, 255), 2)
        #增加時間浮水印
        cv2.putText(frame, datetime.datetime.now().strftime('%A %d %B %Y %I:%M:%S%p'), 
            (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX , 0.35, (0, 0, 255),1) 
        #顯示畫面
        cv2.imshow('Security Feed', frame)
        cv2.imshow('Threshold(foreground mask)', dilate_image)
        cv2.imshow('Frame_delta', frame_delta)
        #關閉快捷鍵
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            cv2.destroyAllWindows()
            break
                
if __name__=='__main__':    
    motion_detection()



email sent!
2020-01-08 12:26:52.752687
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5